<a href="https://colab.research.google.com/github/2714830587-xh/CLRS-GG/blob/master/chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l

     |████████████████████████████████| 82 kB 629 kB/s 
     |████████████████████████████████| 11.2 MB 24.2 MB/s 
     |████████████████████████████████| 61 kB 5.9 MB/s 
     |████████████████████████████████| 9.9 MB 42.3 MB/s 
     |████████████████████████████████| 900 kB 45.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you h

In [ ]:
import torch
import torch.nn as nn


def corr2d(x,k):
  h,w=k.shape
  y=torch.zeros((x.shape[0]-h+1,x.shape[1]-w+1))
  for i in range(y.shape[0]):
    for j in range(y.shape[1]):
      y[i,j]=(x[i:i+h,j:j+w]*k).sum()
  return y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

定义卷积层

In [ ]:
class Con2D(nn.Module):
  def __init__(self,kernal_size):
      super(Con2D,self).__init__()
      self.weight=nn.Parameter(torch.randn(kernal_size))
      self.bias=nn.Parameter(torch.zeros(1))
  def forward(self,x):
    return corr2d(x,self.weight)+self.bias

X = torch.ones((6, 8))
X[:, 2:6] = 0
X
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

学习卷积核

->如果我们只需寻找黑白边缘，那么以上[1, -1]的边缘检测器足以。然而，当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计滤波器。那么我们是否可以学习由X生成Y的卷积核呢？

In [ ]:
Con2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X=X.reshape(1,1,6,8)
Y=Y.reshape(1,1,6,7)
lr=3e-2
for i in range(10):
  output=Con2d(X)
  l=(output-Y)**2
  Con2d.zero_grad()
  l.sum().backward()
  Con2d.weight.data[:]-=lr*Con2d.weight.grad
  if (i+1)%2==0:
    print(f'epoch{i+1},loss{l.sum():.3f}')

epoch2,loss12.204
epoch4,loss2.646
epoch6,loss0.689
epoch8,loss0.216
epoch10,loss0.077


In [ ]:
Con2d.weight.data.reshape(1,2)

tensor([[ 0.9565, -1.0110]])

 **填充和步幅**

 填充可以增加输出的高度和宽度。这常用来使输出与输入具有相同的高和宽。

步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的1/n（n是一个大于1的整数）。

填充和步幅可用于有效地调整数据的维度。

In [ ]:
# 填充
import torch
import torch.nn as nn

def comp_con2d(con2d,x):
  x=x.reshape((1,1)+x.shape)
  y=con2d(x)
  return y.reshape(y.shape[2:])
# con2d=nn.Conv2d(1,1,kernel_size=3,padding=1)
x=torch.randn(size=(8,8))
# comp_con2d(con2d,x).shape

In [ ]:
# 当卷积核的高度和宽度不同时
con2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
#这里的padding数字是指上或左填充的大小,而不是整体,比如当中的2,是上和下都填充2,所以ph=4
comp_con2d(con2d,x).shape

torch.Size([8, 8])

In [ ]:
con2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_con2d(con2d, x).shape

torch.Size([4, 4])

In [ ]:
con2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_con2d(con2d, x).shape


torch.Size([2, 2])

多输入通道

In [ ]:
import torch

import torch
import torch.nn as nn


def corr2d(x,k):
  h,w=k.shape
  y=torch.zeros((x.shape[0]-h+1,x.shape[1]-w+1))
  for i in range(y.shape[0]):
    for j in range(y.shape[1]):
      y[i,j]=(x[i:i+h,j:j+w]*k).sum()
  return y



def corr2d_multi_in(X, K):
  # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
  return sum(corr2d(x, k) for x, k in zip(X, K))
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
# for i,(x, k) in enumerate(zip(X, K)):
#   print(i,x)
#   print(k)
#   break
# print(K.shape)
print(corr2d_multi_in(X, K))

tensor([[ 56.,  72.],
        [104., 120.]])


多输出通道

In [ ]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
print("原始K",K)
K = torch.stack((K, K + 1, K + 2), 0)
K.shape
print(K)


原始K tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])
tensor([[[[0., 1.],
          [2., 3.]],

         [[1., 2.],
          [3., 4.]]],


        [[[1., 2.],
          [3., 4.]],

         [[2., 3.],
          [4., 5.]]],


        [[[2., 3.],
          [4., 5.]],

         [[3., 4.],
          [5., 6.]]]])


In [ ]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1*1 卷积层

??1×1卷积层通常用于调整网络层的通道数量和控制模型复杂性

画一下图,恒容易理解

In [ ]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

池化层

对于给定输入元素，最大汇聚层会输出该窗口内的最大值，平均汇聚层会输出该窗口内的平均值。

汇聚层的主要优点之一是减轻卷积层对位置的过度敏感。

我们可以指定汇聚层的填充和步幅。

使用最大汇聚层以及大于1的步幅，可减少空间维度（如高度和宽度）。

汇聚层的输出通道数与输入通道数相同。

In [5]:
import torch
import torch.nn as nn

def pool2d(x,p_size,mode='max'):
  p_h,p_w=p_size
  y=torch.zeros(x.shape[0]-p_h+1,x.shape[1]-p_w+1)
  for i in range(y.shape[0]):
    for j in range(y.shape[1]):
      if mode=='max':
        y[i,j]=x[i:i+p_h,j:j+p_w].max()
      if mode=='avg':
        y[i,j]=x[i:i+p_h,j:j+p_w].mean()
  return y
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print(pool2d(X, (2, 2)))
pool2d(X, (2, 2), 'avg')

tensor([[4., 5.],
        [7., 8.]])


tensor([[2., 3.],
        [5., 6.]])

填充和步幅

In [9]:
X=torch.arange(16,dtype=torch.float32).reshape(1,1,4,4)
print(x)
pool2d=nn.MaxPool2d(3)
print(pool2d(x))
pool2d1=nn.MaxPool2d(3,padding=1,)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])
tensor([[[[10.]]]])


In [10]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [11]:
X = torch.cat((X, X + 1), 1)
print(X)
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])


tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])